<a href="https://colab.research.google.com/github/tontantip/DADS7202-Deep_learning/blob/main/study_NeuralNetWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Library

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

## Donwload dataset

In [4]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:02<00:00, 9.65MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 149kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 2.78MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 23.6MB/s]


## Setup Batch size

In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


In [6]:
## ตรวจสอบตัวช่วยในการประมวลผลหรือไม่ถ้าไม่มีให้ใช้ cpu
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        ## รีด data จากภาพที่มีขนาด 28*28 pixel ให้เป็น data
        self.flatten = nn.Flatten()
        ## สร้าง layerมาต่อกันตามลำดับ
        self.linear_relu_stack = nn.Sequential(
            ## input layer มี 784 node และออกไป 521 node เป็น hiden layer 1
            nn.Linear(28*28, 512),
            ## hiden layer 1 ใช้ Activation function เป็น ReLU
            nn.ReLU(),
            ## hiden layer 2 มี 512 node และออกไป 512 node เป็น hiden layer 3
            nn.Linear(512, 512),
            ## hiden layer 2 ใช้ Activation function ReLU
            nn.ReLU(),
            ## จาก hiden layer 3 512 node เป็น output layer 10 node (ตามจำนวน class ของ data)
            nn.Linear(512, 10)
        )
    ## กำหนด pipeline ของ Deep learning เริ่มจากการ flatten รีด data จาก 28 * 28 เป็น 784
    ## จากนั้นเรา hidden layer จะได้ output layer ออกมาว่าเป็น class (เป็นตัวเลข 10 ตัว)
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        ## ใช้ return class ออกมา
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300778  [   64/60000]
loss: 2.288687  [ 6464/60000]
loss: 2.271088  [12864/60000]
loss: 2.269110  [19264/60000]
loss: 2.246307  [25664/60000]
loss: 2.224568  [32064/60000]
loss: 2.232948  [38464/60000]
loss: 2.201196  [44864/60000]
loss: 2.194060  [51264/60000]
loss: 2.168505  [57664/60000]
Test Error: 
 Accuracy: 51.1%, Avg loss: 2.155580 

Epoch 2
-------------------------------
loss: 2.163512  [   64/60000]
loss: 2.156447  [ 6464/60000]
loss: 2.095822  [12864/60000]
loss: 2.116573  [19264/60000]
loss: 2.070325  [25664/60000]
loss: 2.007070  [32064/60000]
loss: 2.047525  [38464/60000]
loss: 1.967919  [44864/60000]
loss: 1.973895  [51264/60000]
loss: 1.911473  [57664/60000]
Test Error: 
 Accuracy: 58.0%, Avg loss: 1.896832 

Epoch 3
-------------------------------
loss: 1.932008  [   64/60000]
loss: 1.904789  [ 6464/60000]
loss: 1.779769  [12864/60000]
loss: 1.825720  [19264/60000]
loss: 1.726015  [25664/60000]
loss: 1.666862  [32064/600

In [11]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [12]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [18]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


In [29]:
print(x[[[]]])

tensor([], size=(0, 28, 28))


/tmp/ipython-input-3335849157.py:1: UserWarning: Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /pytorch/torch/csrc/autograd/python_variable_indexing.cpp:345.)
  print(x[[[]]])
